<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/QM_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ib_insync
!pip install ib_insync nest_asyncio numpy scipy matplotlib

# Import necessary libraries
from ib_insync import IB, Stock, MarketOrder
import nest_asyncio
import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt

# Apply the event loop fix for Jupyter
nest_asyncio.apply()

# Initialize IB connection
ib = IB()

# Connect to the IB demo account (make sure TWS is running and API is enabled)
try:
    ib.connect('127.0.0.1', 4002, clientId=2)  # Use port 7497 for TWS Demo
    print("Connected to IB!")
except Exception as e:
    print(f"Failed to connect to IB: {e}")

# Define quantum states for two traders
buy = np.array([1, 0])   # |1> state (buy)
sell = np.array([0, 1])  # |0> state (sell)

# Superposition of buy and sell states
superposition_1 = (1/np.sqrt(2)) * (buy + sell)
superposition_2 = (1/np.sqrt(2)) * (buy - sell)

# Define entanglement operator (e.g., Controlled-NOT gate)
def entanglement_operator():
    CNOT = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 0, 1],
                     [0, 0, 1, 0]])
    return CNOT

# Apply the entanglement operator on the tensor product of the two strategies
def apply_entanglement(state1, state2):
    state_tensor = np.kron(state1, state2)
    return np.dot(entanglement_operator(), state_tensor)

# Apply interference via unitary transformation
def interference_operator(theta):
    return expm(1j * theta * np.array([[0, 1], [1, 0]]))

# Evolve a strategy by applying interference
def evolve_strategy(strategy, theta):
    return np.dot(interference_operator(theta), strategy)

# Initial entangled state of the traders
entangled_state = apply_entanglement(superposition_1, superposition_2)
print("Initial entangled state:\n", entangled_state)

# Define a simple payoff function
def payoff(state1, state2):
    # For simplicity, use dot product of strategies as payoff (in real models, this can be more complex)
    return np.vdot(state1, state2).real

# Simulate the market and optimize player strategies to maximize payoffs
def optimize_payoffs(strategy1, strategy2, steps=10, learning_rate=0.1):
    for step in range(steps):
        # Evolve strategies with small changes (learning rate) to optimize payoffs
        strategy1 = evolve_strategy(strategy1, learning_rate)
        strategy2 = evolve_strategy(strategy2, -learning_rate)  # Players evolve differently

        # Calculate the current payoff
        current_payoff = payoff(strategy1, strategy2)
        print(f"Step {step}: Payoff = {current_payoff}")

    return strategy1, strategy2

# Optimize the strategies
optimized_strategy1, optimized_strategy2 = optimize_payoffs(superposition_1, superposition_2)

# Function to place a market order in the IB Demo account
def place_trade(symbol, action, quantity):
    contract = Stock(symbol, 'SMART', 'USD')
    order = MarketOrder(action, quantity)
    trade = ib.placeOrder(contract, order)
    ib.sleep(1)
    print(f"Trade placed: {action} {quantity} shares of {symbol}")

# Simulate trading based on optimized quantum strategies
def quantum_market_trading(stock_symbol, strategy1, strategy2, steps=10):
    for step in range(steps):
        # Calculate the payoff (interpreted as trade signal strength)
        signal_strength = payoff(strategy1, strategy2)

        # Trade based on the quantum signal
        if signal_strength > 0.5:  # Arbitrary threshold for buy signal
            place_trade(stock_symbol, 'BUY', 10)  # Buy 10 shares
        elif signal_strength < -0.5:  # Arbitrary threshold for sell signal
            place_trade(stock_symbol, 'SELL', 10)  # Sell 10 shares

        # Evolve the strategies for the next step
        strategy1 = evolve_strategy(strategy1, 0.05)
        strategy2 = evolve_strategy(strategy2, -0.05)

# Execute trades in the IB demo account based on quantum strategies
quantum_market_trading('AAPL', optimized_strategy1, optimized_strategy2)

# Disconnect from Interactive Brokers when done
ib.disconnect()
print("Disconnected from IB.")


ERROR:ib_insync.client:API connection failed: ConnectionRefusedError(111, "Connect call failed ('127.0.0.1', 4002)")
ERROR:ib_insync.client:Make sure API port on TWS/IBG is open


Failed to connect to IB: [Errno 111] Connect call failed ('127.0.0.1', 4002)
Initial entangled state:
 [ 0.5 -0.5 -0.5  0.5]
Step 0: Payoff = 0.0
Step 1: Payoff = 0.0
Step 2: Payoff = 0.0
Step 3: Payoff = 0.0
Step 4: Payoff = 0.0
Step 5: Payoff = 0.0
Step 6: Payoff = 0.0
Step 7: Payoff = 0.0
Step 8: Payoff = 0.0
Step 9: Payoff = 0.0
Disconnected from IB.
